In [1]:
import re
import tqdm
import numpy as np
import pandas as pd
from tqdm import tqdm as tq
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
alldata = pd.read_csv("C:/Users/wjddb/LotteMembers/아카이브/all.csv")
alldata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4630047 entries, 0 to 4630046
Data columns (total 21 columns):
 #   Column         Dtype  
---  ------         -----  
 0   고객번호           object 
 1   지역             object 
 2   성별나이           object 
 3   영수증번호          object 
 4   온오프            int64  
 5   제휴사            object 
 6   거래날짜           object 
 7   거래시간           int64  
 8   구매금액           float64
 9   구매수량           float64
 10  상품이름           object 
 11  상품대분류          object 
 12  상품소분류          object 
 13  회원유무           int64  
 14  상품클래스          object 
 15  거래요일           int64  
 16  R_Quantile     int64  
 17  F_Quantile     int64  
 18  M_Quantile     int64  
 19  RFMClass       int64  
 20  RFMClassScore  int64  
dtypes: float64(2), int64(9), object(10)
memory usage: 741.8+ MB


In [32]:
# RFMClassScore 가 18인 고객들만을 통한 연관분석 실행.

df_tmp = alldata[['영수증번호','상품클래스','RFMClassScore']].sort_values(by="영수증번호").reset_index(drop=True)
df_tmp = df_tmp[df_tmp['RFMClassScore'] == 18 ]


#df_tmp = alldata[['영수증번호','상품클래스']]
df_tmp = df_tmp.sort_values('영수증번호').reset_index(drop=True)

relist = []
# 영수증번호의 sorting을 위해 안에있는 문자열 제거.

letterlist = ["M","A","B","C","D","E"]
for i in df_tmp['영수증번호']:
    i = re.sub("M|C|D|E|B|A","",i)
    relist.append(int(i))
    
df_tmp['영수증번호'] = relist
df_tmp.영수증번호

0         20122911011862577
1         20123017011030664
2         20123022013121852
3         20123111010929930
4         20123112010615781
                ...        
952368           6052119978
952369           6052119978
952370           6052119978
952371           6052119978
952372           6052119978
Name: 영수증번호, Length: 952373, dtype: int64

In [33]:
# df_tmp['영수증번호']
#df_tmp = df_tmp.sort_values("영수증번호").reset_index(drop=True)
#df_tmp = df_tmp.sample(frac = 0.2)

# 총 데이터양과 고객의 수를 체크.
df_tmp = df_tmp.sort_values("영수증번호").reset_index(drop=True)
print(len(df_tmp.영수증번호.drop_duplicates()))
print(len(df_tmp))

297701
952373


In [34]:

# 영수증 별 상품을 뽑기위해 인덱스를 생성.
index_list = []
numb = 0

for i in tq(range(len(df_tmp.영수증번호)-1)): 
        if df_tmp.영수증번호[i] == df_tmp.영수증번호[i+1]:
            index_list.append(numb)
        else:
            numb += 1
            index_list.append(numb)

# 리스트 크기 맞추기.    
index_list.insert(-1,len(df_tmp.영수증번호))
df_tmp['인덱스'] = index_list
df_tmp

100%|███████████████████████████████████████████████████████████████████████| 952372/952372 [00:25<00:00, 37191.51it/s]


,영수증번호,상품클래스,RFMClassScore,인덱스
0,1000002265,생활,18,1
1,1000006315,스포츠,18,2
2,1000011560,스포츠,18,3
3,1000011639,스포츠,18,3
4,1000011639,스포츠,18,4
...,...,...,...,...
952368,21123120025223019,기타,18,297697
952369,21123121011172543,기타,18,297698
952370,21123121014718098,기타,18,297699
952371,21123121014811379,기타,18,952373


In [35]:
# 구매물품 별 리스트 뽑기
df_tmp_arr=[[]for i in range(len(df_tmp.인덱스)+1)]
# df_tmp_arr = []
num=0
for i in tq(df_tmp['상품클래스']) :
    df_tmp_arr[df_tmp['인덱스'][num]].append(i)
    num+=1
    
df_tmp_arr

100%|██████████████████████████████████████████████████████████████████████| 952373/952373 [00:08<00:00, 111419.64it/s]


[[],
 ['생활'],
 ['스포츠'],
 ['스포츠', '스포츠'],
 ['스포츠'],
 ['패션', '스포츠'],
 ['스포츠'],
 ['패션'],
 ['생활'],
 ['인테리어'],
 ['패션', '식품', '식품'],
 ['식품'],
 ['식품'],
 ['패션'],
 ['식품'],
 ['식품'],
 ['패션'],
 ['패션'],
 ['유아'],
 ['식품'],
 ['유아', '식품', '식품'],
 ['패션'],
 ['식품'],
 ['유아'],
 ['패션'],
 ['인테리어'],
 ['유아', '패션', '패션', '패션', '가전', '패션'],
 ['패션'],
 ['패션'],
 ['패션', '스포츠'],
 ['스포츠'],
 ['유아'],
 ['패션', '패션'],
 ['패션', '패션'],
 ['패션'],
 ['패션'],
 ['패션'],
 ['스포츠'],
 ['인테리어', '스포츠', '스포츠', '스포츠'],
 ['스포츠'],
 ['인테리어'],
 ['인테리어'],
 ['인테리어'],
 ['인테리어', '인테리어'],
 ['인테리어'],
 ['패션'],
 ['건강'],
 ['생활'],
 ['패션', '패션', '패션', '패션', '패션', '패션', '패션', '패션', '패션', '패션'],
 ['패션'],
 ['건강'],
 ['유아'],
 ['패션'],
 ['식품'],
 ['유아'],
 ['패션'],
 ['유아'],
 ['패션'],
 ['패션'],
 ['인테리어'],
 ['스포츠'],
 ['패션', '패션', '패션', '패션', '패션'],
 ['패션'],
 ['인테리어'],
 ['패션'],
 ['유아'],
 ['패션'],
 ['인테리어'],
 ['인테리어'],
 ['식품'],
 ['패션'],
 ['식품'],
 ['스포츠'],
 ['패션', '패션'],
 ['패션'],
 ['패션'],
 ['스포츠'],
 ['유아'],
 ['패션', '패션'],
 ['패션'],
 ['패션', '인테리어', '인테리어', '인테리어', '인테리어', '인테리

# Original

In [ ]:
# from tqdm import tqdm
# x_sample = rnp.sample(1000).reset_index(drop=True)
# x=[[] in range(len(rnp))]

# for i in tqdm(range(len(x_sample))):
#     if len(x_sample.loc[no_dups== x_sample['영수증번호'][i]]) >  1:
#         r = x_sample.loc[no_dups== x_sample['영수증번호'][i]]
#         r_names = r['상품이름']
#         a = []
#         for j in r_names:
#             a.append([j])
#         x.append(a)
#     else:
#         x.append([x_sample.상품이름[i]])

# 연관분석 모델.

In [36]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
df_tmp_arr.pop(0)
te = TransactionEncoder()
x_ary = te.fit(df_tmp_arr).transform(df_tmp_arr)
df = pd.DataFrame(x_ary, columns=te.columns_)

df

,가전,건강,기타,도서/취미,문구,생활,스포츠,식품,유아,인테리어,주류,패션
0,False,False,False,False,False,True,False,False,False,False,False,False
1,False,False,False,False,False,False,True,False,False,False,False,False
2,False,False,False,False,False,False,True,False,False,False,False,False
3,False,False,False,False,False,False,True,False,False,False,False,False
4,False,False,False,False,False,False,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
952368,False,False,False,False,False,False,False,False,False,False,False,False
952369,False,False,False,False,False,False,False,False,False,False,False,False
952370,False,False,False,False,False,False,False,False,False,False,False,False
952371,False,False,False,False,False,False,False,False,False,False,False,False


**SUPPORT**

In [37]:
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.043402,(기타)
1,0.010631,(문구)
2,0.038334,(생활)
3,0.013230,(스포츠)
4,0.195695,(식품)
5,0.013044,(유아)
6,0.031189,(주류)
7,0.058722,(패션)
8,0.033212,"(식품, 생활)"
9,0.023682,"(식품, 주류)"


**RESULT**

In [41]:
ar = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01).sort_values(by = ['antecedents', 'consequents', 'confidence'])

In [39]:
ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(식품),(생활),0.195695,0.038334,0.033212,0.169712,4.427215,0.025710,1.158232
2,(식품),(주류),0.195695,0.031189,0.023682,0.121014,3.879967,0.017578,1.102191
5,(식품),(패션),0.195695,0.058722,0.019911,0.101746,1.732688,0.008420,1.047898
1,(생활),(식품),0.038334,0.195695,0.033212,0.866385,4.427215,0.025710,6.019589
3,(주류),(식품),0.031189,0.195695,0.023682,0.759292,3.879967,0.017578,3.341408
4,(패션),(식품),0.058722,0.195695,0.019911,0.339079,1.732688,0.008420,1.216945
